In [1]:
import numpy as np, glob, time, shutil, string, os, joblib, datetime
import sys, os, numpy as np
from datetime import datetime
import pickle, cv2, pims, tqdm
from IPython.display import clear_output
from PIL import Image
%matplotlib inline

In [1]:
import shutil, os, glob

In [5]:
model_name = 'i3d_resnet50_v1_kinetics400'
feat_file = r"F:/S-Home/r2/data/features/my_dataset/video\notPhysicalViolence\1026.Vanilla.npy"
feat_file = feat_file.replace('/video', f"/video/{model_name.split('_')[0]+str(10)}s/")
feat_file  

'F:/S-Home/r2/data/features/my_dataset/video/i3d10s/\\notPhysicalViolence\\1026.Vanilla.npy'

In [2]:
import tensorflow as tf

In [3]:
import glob, decord
from decord import VideoReader
from decord import cpu, gpu
decord.bridge.set_bridge('tensorflow')

In [4]:
from tensorflow.keras.applications.resnet import preprocess_input
backbone = tf.keras.applications.ResNet50(include_top=False,weights="imagenet",pooling='avg')

In [4]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
backbone = tf.keras.applications.InceptionV3(include_top=False,weights="imagenet",pooling='avg')

In [5]:
def ExtractFeature(FileName,frames = 16, width=224, height=224):
    
    V = pims.Video(FileName) 
    duration = len(V) 
      
    try:    frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=False))
    except: frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=True))
    
    Frames = np.array(V[frame_id_list])
    Frames = np.array([cv2.resize(frame,(width,height)) for frame in Frames])
    features = np.array(backbone.predict(preprocess_input(Frames)))
    return features


def ExtractFeatureDecord(FileName,frames = 16, width=224, height=224):
    
    try:
        V = VideoReader(FileName, width, height)
        duration = len(V)
        try:    frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=False))
        except: frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=True))
        
        Frames = V.get_batch(frame_id_list).asnumpy()
        features = np.array(backbone.predict(preprocess_input(Frames)))
    
    except:return ExtractFeature(FileName,frames,width,height)
    return features

def ExtractFeatureCV2(FileName,frames = 16, width=224, height=224):
    """get frames from video using cv2"""
    V = cv2.VideoCapture(FileName)
    duration = int(V.get(cv2.CAP_PROP_FRAME_COUNT))
    try:    frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=False))
    except: frame_id_list = np.sort(np.random.choice(range(duration), frames, replace=True))
    
    Frames = np.array([V.read()[1] for i in frame_id_list])
    Frames = np.array([cv2.resize(frame,(width,height)) for frame in Frames])
    features = np.array(backbone.predict(preprocess_input(Frames)))
    V.release()
    return features
    

In [6]:
data  = glob.glob(r'H:\S-Home\RecognizingPhysicalViolence\dataset\rlvs\**\*.avi',recursive=True)
#data.extend(glob.glob(r'H:\S-Home\RecognizingPhysicalViolence\dataset\my_dataset\**\*.avi',recursive=True))
#data.extend(glob.glob(r'H:\S-Home\RecognizingPhysicalViolence\dataset\rlvs\**\*.avi',recursive=True))
data.extend(glob.glob(r'H:\S-Home\RecognizingPhysicalViolence\dataset\rlvs\**\*.mp4',recursive=True))
len(data)

2000

In [7]:
#os.makedirs(r'video_features\ResNet50\notPhysicalViolence',exist_ok=True)
failed =[]
for i in tqdm.tqdm(data[:]): 
    try:
        try:features = ExtractFeatureCV2(i)
        except: features = ExtractFeatureDecord(i)
        name = i.replace(i[-4:],'.npy')
        
        #sname = "video_features\ResNet50\\notPhysicalViolence\{}.npy".format(name)
        #print(sname)
        np.save(name,features,allow_pickle=True)
    except:
        failed.append(i)

100%|██████████| 2000/2000 [09:10<00:00,  3.63it/s]


In [8]:
failed

[]

In [12]:
data  = glob.glob(r'dataset\PhysicalViolence\*.mp4')
print(len(data))
os.makedirs(r'video_features\ResNet50\PhysicalViolence',exist_ok=True)

for i in tqdm.tqdm(data): 
    try:features = ExtractFeatureDecord(i)
    except: features = ExtractFeatureCV2(i)
    name = i.split('\\')[-1][:-4]
    sname = "video_features\ResNet50\PhysicalViolence\{}.npy".format(name)
    #print(sname)
    np.save(sname,features,allow_pickle=True)

614


100%|██████████| 614/614 [18:52<00:00,  1.85s/it]


In [1]:

%reset -f
import gc
import tensorflow_hub as hub,tensorflow as tf
import tensorflow_io as tfio
import tqdm,numpy as np,os,tqdm,glob
from IPython.display import clear_output


yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

@tf.function
def load_wav_16k_mono(filename):
    """ read in a waveform file and convert to 16 kHz mono """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

start = 600
#class_ = "notPhysicalViolence"
audio_data = glob.glob(r'H:\S-Home\Home Office\Violence\archive\v\*.wav')
os.makedirs(r"H:\S-Home\Home Office\Violence\archive\embedding" , exist_ok=True)  

'''import multiprocessing
p = multiprocessing.Pool(3)
embeddingss = p.map(save_embedding, audio_data[:6])'''


def save_embedding(audio_path):
    """ save the embedding of the audio file """
    gg, embeddings, kk = yamnet_model(load_wav_16k_mono(audio_path))  
    name = audio_path.split('\\')[-1][:-4]+'.wav'
    
    sname = os.path.join(r'H:\S-Home\Home Office\Violence\archive\embedding',name[:-4]+'.npy')
    print(sname)
    embeddings = np.max(embeddings,axis=0)*.25+np.mean(embeddings,axis=0)*.75 
    embeddings = embeddings.reshape(-1)
    print(embeddings.shape)

    np.save(sname,embeddings , allow_pickle=True)
    
    
for i in tqdm.tqdm(audio_data[start:]):
    save_embedding(i)
    clear_output()

100%|██████████| 338/338 [01:48<00:00,  3.10it/s]


In [6]:
failed = np.load('failed.npy',allow_pickle=True)

In [20]:
import tqdm,numpy as np,os,tqdm,glob
import moviepy.editor as mpy
from IPython.display import clear_output

def video_to_wav(video):
    """convert video to wav using moviepy and save it in audio directory"""
    if video.endswith('.wav'):return
    clip = mpy.VideoFileClip(video)
    audio = clip.audio
    path = video[:-4] + ".wav"
    audio.write_audiofile(path)

In [9]:
def ifFileExist(file):
    """check if file exists"""
    if os.path.exists(file):return True
    else:return False

In [10]:
data = glob.glob(r"H:\S-Home\Home Office\Violence\archive\Real Life Violence Dataset\*\*.avi", recursive=True)
data.extend(glob.glob(r"H:\S-Home\Home Office\Violence\archive\Real Life Violence Dataset\*\*.mp4", recursive=True))
len(data)

2000

In [11]:
for i in tqdm.tqdm(data): 
    name = i.split('\\')[-1][:-4]
    sname = "H:\S-Home\Home Office\Violence\\archive\\v2\{}.npy".format(name)
    if ifFileExist(sname):continue
    try:features = ExtractFeatureDecord(i)
    except: features = ExtractFeatureCV2(i)

    #print(sname)
    np.save(sname,features,allow_pickle=True)

100%|██████████| 2000/2000 [00:51<00:00, 38.89it/s] 


In [ ]:
def get_model(dimension = 512,summary = False,input_shape = INPUT_DIM):
    
    FeatureInput = Input(shape=input_shape)
    Average_feature = layers.Flatten()(FeatureInput) if  IF_MXNET_MODEL else tf.reduce_max(FeatureInput,-2)*.25 + tf.reduce_mean(FeatureInput,-2)*.75
    
    AudioInput =   Input(shape=(1024))
    audio_ = layers.Dense(dimension//4,kernel_initializer='normal', activation='linear')(AudioInput)
    audio_ = layers.BatchNormalization()(audio_)
    audio_ = layers.Activation(tf.nn.relu)(audio_)
    audio_ = layers.Dropout(.5)(audio_)
    
    video_ = layers.Flatten()(Average_feature)
    video_ = layers.Dense(dimension//2,kernel_initializer='normal', activation='linear')(video_)
    video_ = layers.BatchNormalization()(video_)
    video_ = layers.Activation(tf.nn.relu)(video_)
    video_ = layers.Dropout(.5)(video_)
    
    combined = layers.concatenate([video_,audio_],axis=1)
    
    output = layers.Dense(1,kernel_initializer='normal', activation='sigmoid',name = 'classifier')(combined)
    
    model = Model(inputs= [FeatureInput,AudioInput], outputs = output)
    model.compile(optimizer=tf.optimizers.Nadam(learning_rate=0.001), loss='binary_crossentropy',metrics=['accuracy'])
    
    if summary:print(model.summary())
    
    return model

In [ ]:
import glob
print(glob.glob('abc'))

: 

In [1]:
import librosa

In [16]:
import requests
import io
import speech_recognition as sr
from pydub import AudioSegment


# This function translate speech to text
def speech_to_text(file):
    recognizer = sr.Recognizer()
    audio = sr.AudioFile(file)
    with audio as source:
        speech = recognizer.record(source)
        try:
            # Call recognizer with audio and language
            text = recognizer.recognize_google(speech, language='pt-BR')
            print("Você disse: " + text)
            return text
        # If recognizer don't understand
        except:
            print("Não entendi")

def mp4_to_wav(file):
    audio = AudioSegment.from_file(file, format="mp4")
    audio.export("audio.wav", format="wav")
    return audio

def mp4_to_wav_mem(file):
    audio = AudioSegment.from_file_using_temporary_files(file, 'mp4')
    file = io.BytesIO()
    file = audio.export(file, format="wav")
    file.seek(0)
    return file

In [2]:
audio_path = r"H:\S-Home\RecognizingPhysicalViolence\dataset\my_dataset\video\PhysicalViolence\3017.mp4"
y, sr = librosa.load(audio_path)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

D:\Program\Anaconda\envs\tf26\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NoBackendError: 

In [15]:
audio_path = r"H:\S-Home\RecognizingPhysicalViolence\test_data\2009.mp4"

file = open(audio_path, 'rb').read()

file = io.BytesIO(file)
audio_only = mp4_to_wav_mem(file)

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
from os import path
from pydub import AudioSegment

# files                                                                         
src = "transcript.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

In [ ]:
def video_to_audio(video_path,audio_path):
    """convert video to audio using ffmpeg"""
    import subprocess
    subprocess.call(['ffmpeg', '-i', video_path, '-f', 'wav', audio_path])

In [17]:
import librosa

audio_path = r"H:\S-Home\RecognizingPhysicalViolence\dataset\my_dataset\audio\PhysicalViolence\3017.wav"
y, sr = librosa.load(audio_path)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

In [19]:
mfcc.shape

(20, 488)

In [3]:
import decord

help(decord.video_loader)

Help on module decord.video_loader in decord:

NAME
    decord.video_loader - Video Loader.

CLASSES
    builtins.object
        VideoLoader
    
    class VideoLoader(builtins.object)
     |  VideoLoader(uris, ctx, shape, interval, skip, shuffle, prefetch=0)
     |  
     |  Multiple video loader with advanced shuffling and batching methods.
     |  
     |  Parameters
     |  ----------
     |  uris : list of str
     |      List of video paths.
     |  ctx : decord.Context or list of Context
     |      The context to decode the video file, can be decord.cpu() or decord.gpu().
     |      If ctx is a list, videos will be evenly split over many ctxs.
     |  shape : tuple
     |      Returned shape of the batch images, e.g., (2, 320, 240, 3) as (Batch, H, W, 3)
     |  interval : int
     |      Intra-batch frame interval.
     |  skip : int
     |      Inter-batch frame interval.
     |  shuffle : int
     |      Shuffling strategy. Can be
     |      `0`:  all sequential, no seekin